# Weekly Tracker Function

The objective of this function is to create tabular data used for creating a Weekly Sales Tracker. Note there is a function for quarterly info, and a separate one for annual.

Tableau Server Team/Rep Performance Sales Table is downloaded and fed into this function. It takes in current number of billing days, and today's date. Quarterly also takes in quarter number, and yearly takes in total number of billing days (251 in 2022).

**Note:** Starting Q2, this dataframe will also include all NTX territory.

The function knows the finalized targets for each team, and uses this in conjunction with the imported sales data to create a dataframe; it also automatically performs calculations for all columns requested:
   * QTD/YTD Target
   * % to QTD/YTD Target 
   * % Growth PY
   * % to Quarter/Annual Target
    
The function automatically displays the table it has created, as well as download that table as a CSV file to same path as this notebook. 

From there, the Power BI file is refreshed with the new data source.

## Data Acquisition

**Team/Rep Sales Performance**

Explore / Territory Leadership / West Zone Daily Sales (2022) Territory / Team/Rep Performance / 
Download Team_Rep Sales Table_crosstab

Following filters:
* QTD or YTD 
* Financial Sales
* Product Groups: CEMENT, EXTREMITIES, FOOT & ANKLE, HIPS, KNEES, RESTORATIVE THERAPY, SPORTS MEDICINE, TRAUMA
* Team/Rep Name: JOHN KOEHLER, MATT BORROR, RYAN FURNISH, TEAM BLAKE ROBERTS, TEAM DOSSIE LAYTON, TEAM LUNA, TEAM MIDAS, TEAM NEO, TEAM OKC, TEAM RICK ATCHISON, TEAM SEVE SANCHEZ, TEAM SOUTHWEST, TEAM TULSA
* Direct Comparison and 2021 as PY
* Today's date selected; currently updates are occurring on Friday mornings.

.txt file converted to .csv UTF-8


## Quarter Dashboard

In [1]:
import pandas as pd
import numpy as np

def quarterly_dashboard(tableau_file_name, q_current_billing_days, q_number, date):
    
    quarter_billing_days = {1:63,2:63,3:64,4:61}
    #Date is written mmddyy
    
    #Establish each team and their finalized target numbers for the quarter
    qtd_targets = [['JOHN KOEHLER',18838.00],['MATT BORROR',136057.00], ['RYAN FURNISH', 571732.00],
           ['TEAM BLAKE ROBERTS', 293321.00], ['TEAM DOSSIE LAYTON', 1735376.00], ['TEAM LUNA', 1565876.00],
           ['TEAM MIDAS', 2595550.00], ['TEAM NEO', 949048.00], ['TEAM OKC', 2478139.00], 
           ['TEAM RICK ATCHISON', 2391345.00], ['TEAM SEVE SANCHEZ', 433499.00], ['TEAM SOUTHWEST', 1257707.00], 
           ['TEAM TULSA', 1301747.00]]

    qtd_tracker = pd.DataFrame(qtd_targets, columns = ['team_rep_name', 'quarter_target'])
    
    #Add column to quarter to date target
    qtd_tracker['qtd_target'] = ((abs(qtd_tracker['quarter_target']))/quarter_billing_days[q_number])*q_current_billing_days
    
    # Import Tableau Team/Rep Sales Performance
    qtd_df = pd.read_csv(tableau_file_name)
    # Clean dataframe
    qtd_df = qtd_df[qtd_df['Product Group']=='Total']
    qtd_df = qtd_df[['Team/Rep Name','Current Sales', 'Previous Sales']]
    qtd_df.rename(columns={'Team/Rep Name': 'team_rep_name', 'Current Sales': 'current_sales', 'Previous Sales': 'sales_2021'},
                  inplace=True)
    
    qtd_df['current_sales'] = qtd_df['current_sales'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(int)
    qtd_df['sales_2021'] = qtd_df['sales_2021'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(int)
    
    # Join Tableau dataset to the teams' targets dataframe
    qtr_merged = pd.merge(qtd_tracker, qtd_df, how='inner', on='team_rep_name')

    
    # Add calculated fields: % to qtd target, % growth from previous year, % to quarter target
    qtr_merged['percent_to_qtd_target'] = qtr_merged['current_sales']/qtr_merged['qtd_target']
    qtr_merged['percent_growth_py'] = abs(qtr_merged['current_sales']-qtr_merged['sales_2021']) / qtr_merged['sales_2021']
    qtr_merged['percent_to_qtr_target'] = abs(qtr_merged['current_sales']/qtr_merged['quarter_target'])
    # Identifier for Specialty Reps
    qtr_merged.iloc[:3,0] = qtr_merged.iloc[:3,0]+'*'
    
    qtr_merged.to_csv('quarterly_dash_'+date+'.csv', index=False)
    
    return qtr_merged
    

In [2]:
quarterly_dashboard('quarter_tr.csv',38,1,'022522')

,team_rep_name,quarter_target,qtd_target,current_sales,sales_2021,percent_to_qtd_target,percent_growth_py,percent_to_qtr_target
0,JOHN KOEHLER*,18838.0,1.136260e+04,7831,10976,0.689191,0.286534,0.415702
1,MATT BORROR*,136057.0,8.206613e+04,53341,48919,0.649976,0.090394,0.392049
2,RYAN FURNISH*,571732.0,3.448542e+05,284615,262021,0.825320,0.086230,0.497812
3,TEAM BLAKE ROBERTS,293321.0,1.769238e+05,133901,107203,0.756829,0.249042,0.456500
4,TEAM DOSSIE LAYTON,1735376.0,1.046735e+06,593921,632838,0.567404,0.061496,0.342243
5,TEAM LUNA,1565876.0,9.444966e+05,795777,1040367,0.842541,0.235100,0.508199
6,TEAM MIDAS,2595550.0,1.565570e+06,1341533,1649560,0.856898,0.186733,0.516859
7,TEAM NEO,949048.0,5.724417e+05,582097,536709,1.016867,0.084567,0.613348
8,TEAM OKC,2478139.0,1.494751e+06,1335902,1314603,0.893729,0.016202,0.539075
9,TEAM RICK ATCHISON,2391345.0,1.442399e+06,1202916,1413143,0.833969,0.148766,0.503029


## Annual Dashboard

In [3]:
#import pandas as pd
#import numpy as np

def yearly_dashboard(tableau_file_name, y_current_billing_days, y_total_billing_days, date):

    #Date is written mmddyy
    
    #Establish each team and their finalized target numbers for the quarter
    ytd_targets = [['JOHN KOEHLER',111526.00],['MATT BORROR',663100.00], ['RYAN FURNISH', 2671430.00],
                   ['TEAM BLAKE ROBERTS', 1241576.00], ['TEAM DOSSIE LAYTON', 6029345.00], ['TEAM LUNA', 5236744.00],
                   ['TEAM MIDAS', 13060458.00], ['TEAM NEO', 3432551.00], ['TEAM OKC', 9145078.00], 
                   ['TEAM RICK ATCHISON', 11185436.00], ['TEAM SEVE SANCHEZ', 1763278.00], ['TEAM SOUTHWEST', 3976790.00],
                   ['TEAM TULSA', 5627742.00]]

    ytd_tracker = pd.DataFrame(ytd_targets, columns = ['team_rep_name', 'year_target'])
    
    #Add column to quarter to date target
    ytd_tracker['ytd_target'] = (abs(ytd_tracker['year_target'])/y_total_billing_days)*y_current_billing_days
    
    # Import Tableau Team/Rep Sales Performance
    ytd_df = pd.read_csv(tableau_file_name)
    
    # Clean dataframe
    ytd_df = ytd_df[ytd_df['Product Group']=='Total']
    ytd_df = ytd_df[['Team/Rep Name', 'Current Sales', 'Previous Sales']]
    ytd_df.rename(columns={'Team/Rep Name': 'team_rep_name', 'Current Sales':'current_sales',
                           'Previous Sales': 'sales_2021'}, inplace=True)
    ytd_df['current_sales'] = ytd_df['current_sales'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(int)
    ytd_df['sales_2021'] = ytd_df['sales_2021'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(int)
    
    # Join Tableau dataset to the teams' targets dataframe
    year_merged = pd.merge(ytd_tracker, ytd_df, how='inner', on='team_rep_name')

    
    # Add calculated fields: % to qtd target, % growth from previous year, % to quarter target
    year_merged['percent_to_ytd_target'] = abs(year_merged['current_sales']/year_merged['ytd_target'])
    year_merged['percent_growth_py'] = abs((year_merged['current_sales']-year_merged['sales_2021'])/year_merged['sales_2021'])
    year_merged['percent_to_annual_target'] = abs(year_merged['current_sales']/year_merged['year_target'])
    # Identifier for Specialty Reps
    year_merged.iloc[:3,0] = year_merged.iloc[:3,0]+'*'
    
    year_merged.to_csv('yearly_dash_'+date+'.csv', index=False)
    
    return year_merged

In [4]:
yearly_dashboard('year_tr.csv',38,251, '022522')

,team_rep_name,year_target,ytd_target,current_sales,sales_2021,percent_to_ytd_target,percent_growth_py,percent_to_annual_target
0,JOHN KOEHLER*,111526.0,1.688441e+04,7831,10976,0.463801,0.286534,0.070217
1,MATT BORROR*,663100.0,1.003896e+05,53341,48919,0.531340,0.090394,0.080442
2,RYAN FURNISH*,2671430.0,4.044396e+05,284615,262021,0.703727,0.086230,0.106540
3,TEAM BLAKE ROBERTS,1241576.0,1.879677e+05,133901,107203,0.712362,0.249042,0.107848
4,TEAM DOSSIE LAYTON,6029345.0,9.128092e+05,593921,632838,0.650652,0.061496,0.098505
5,TEAM LUNA,5236744.0,7.928138e+05,795777,1040367,1.003738,0.235100,0.151960
6,TEAM MIDAS,13060458.0,1.977280e+06,1341533,1649560,0.678474,0.186733,0.102717
7,TEAM NEO,3432551.0,5.196691e+05,582097,536709,1.120130,0.084567,0.169581
8,TEAM OKC,9145078.0,1.384514e+06,1335902,1314603,0.964889,0.016202,0.146079
9,TEAM RICK ATCHISON,11185436.0,1.693413e+06,1202916,1413143,0.710350,0.148766,0.107543


*Dev Notes*

* str.replace updating soon; must include regex=True [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html#pandas.Series.str.replace)

* targets must be updated for each new quarter